> #### wanglijie-19大数据-期末

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 1、导入相应包及数据

In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf
import time

# 导入用于提交预测结果的包
INPUT_DIR = '../input/tensorflow-great-barrier-reef'
sys.path.insert(0, INPUT_DIR)
import greatbarrierreef

### 2、将TensorFlow COTS检测模型加载到内存中，并定义一些用于运行的util函数推论。



In [ ]:
MODEL_DIR = '../input/cots-detection-w-tensorflow-object-detection-api/cots_efficientdet_d0'
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn_tf_odt = tf.saved_model.load(os.path.join(os.path.join(MODEL_DIR, 'output'), 'saved_model'))
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

In [ ]:
def load_image_into_numpy_array(path):
    
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(io.BytesIO(img_data))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

##### ***加载图片到numpy_array***

*将图像放入Numpy数组以输入TensorFlow图。*
*注意，按照惯例，我们将其放入有形的numpy数组中，(高度，宽度，通道)，其中通道=3表示RGB。*  

*路径：文件路径(可以是本地的，也可以是colossus架构中的)。*

*返回值：uint8 numpy数组形状（高度，宽度，3）。*


In [ ]:
def detect(image_np):

    input_tensor = np.expand_dims(image_np, 0)
    start_time = time.time()
    detections = detect_fn_tf_odt(input_tensor)
    return detections
    

##### *从numpy图像识别COTS*

### 3、运行结果并构建提交数据

In [ ]:
env = greatbarrierreef.make_env()   # 初始化环境
iter_test = env.iter_test()    # 循环测试集和样本提交的迭代器

In [ ]:
DETECTION_THRESHOLD = 0.19

submission_dict = {
    'id': [],
    'prediction_string': [],
}

for (image_np, sample_prediction_df) in iter_test:
    height, width, _ = image_np.shape
    
    # Run object detection using the TensorFlow model.
    detections = detect(image_np)
    
    # Parse the detection result and generate a prediction string.
    num_detections = detections['num_detections'][0].numpy().astype(np.int32)
    predictions = []
    for index in range(num_detections):
        score = detections['detection_scores'][0][index].numpy()
        if score < DETECTION_THRESHOLD:
            continue

        bbox = detections['detection_boxes'][0][index].numpy()
        y_min = int(bbox[0] * height)
        x_min = int(bbox[1] * width)
        y_max = int(bbox[2] * height)
        x_max = int(bbox[3] * width)
        
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    # Generate the submission data.
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

    print('Prediction:', prediction_str)   